<a href="https://colab.research.google.com/github/guptaharsh-in/RSI_Trading_Bot_Using_Binance_WebSocket/blob/main/RSI_Trading_Bot_Using_Binance_WebSocket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install and Import Talib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib
import talib

In [ ]:
# Install and Import Websocket Client
pip install websocket-client

In [159]:
# Import other Libraries
import talib
import websocket
import requests
import pandas as pd
import numpy as np
import json
import ssl
     

In [ ]:
# Connect to Binance Websocket
cc= 'btcusd'
interval = "1m" 
socket = f"wss://stream.binance.com:9443/ws/{cc}t@kline_{interval}"

In [ ]:
# Trading Strategy Parameters
amount=1000000
money_remaining = 1000000
portfolio = 0
investment, closes, highs, lows = [], [], [], []
buy_price = []
sell_price = []
rsi_signal = []
signal = 0

In [ ]:
# Paper Trading Simulation Function
def buy(allocated_money, price):
    global closes,highs,lows, money_remaining, portfolio, investment, signal
    quantity = allocated_money  / price
    money_remaining -= quantity*price
    portfolio += quantity 
    if investment == []:
      investment.append(allocated_money)
    else:
      investment.append(allocated_money)
      investment[-1] += investment[-2]

def sell(allocated_money, price):
    global closes,highs,lows, money_remaining, portfolio, investment, signal
    quantity = allocated_money  / price
    money_remaining += quantity*price
    portfolio -= quantity 
    investment.append(-allocated_money)
    investment[-1] += investment[-2]
      
    


In [ ]:
# Fetch Live Data using Websocket Client
def on_close(ws):
  print("Close")

def on_message(ws, message):
  global closes,highs,lows, money_remaining, portfolio, investment, signal, position
  json_message = json.loads(message)
  cs = json_message["k"]
  candle_closed, close, high, low = cs["x"], cs["c"],cs["h"],cs["l"]


  if candle_closed:
    closes.append(float(close))
    highs.append(float(high))
    lows.append(float(low))
    last_price = closes[-1]
    print(f'Closes : {closes[-5:]}')

# RSI Strategy Using Ta-lib

    rsi = talib.RSI(np.array(closes), timeperiod=3)
    last_rsi=round(rsi[-1],2)
    print(last_rsi)
 
    if (rsi[-1] > 40 and rsi < 40).any():
              if signal != 1:
                buy_price.append(closes)
                sell_price.append(np.nan)
                signal = 1
                rsi_signal.append(signal)
              else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                rsi_signal.append(0)
    elif (rsi[-1] < 60 and rsi > 60).any():
              if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(closes)
                signal = -1
                rsi_signal.append(signal)
              else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                rsi_signal.append(0)
    else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                rsi_signal.append(0)

    print(rsi_signal[-5:])

     
# Generate Signal and Trade
    if rsi_signal[-1] == 1:
           buy(250000, last_price)
           print(f'(Bitcoin Bought ="{portfolio}")')
    elif rsi_signal[-1] == -1:
           sell(250000, last_price)
           print(f'(Bitcoin Sold ="{portfolio}")')
    else:
           print("No Trade")



wsapp = websocket.WebSocketApp(socket, on_message = on_message, on_close = on_close)

In [ ]:
# Real Time Trading Bot
wsapp.run_forever()